In [31]:
import numpy as np
import matplotlib.pyplot as plt
import datajoint as dj
import pandas as pd
import pandas as pd
from pathlib import Path
# import logging
# log = logging.getLogger()
# log.setLevel(logging.DEBUG)

from io_utils import *
from peh_utils import * 
%load_ext autoreload
%autoreload 2
dj.blob.use_32bit_dims = True # necessary for pd.blob read

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
#### MULTI SESSION


animal_id = 'R600'
bdata = dj.create_virtual_module("new_acquisition", "bdatatest")
subject_session_key = {"ratname": animal_id}

pd_blobs = (bdata.Sessions & subject_session_key).fetch(
    "protocol_data", as_dict=True
)

sess_ids, dates, trials = (bdata.Sessions & subject_session_key).fetch(
    "sessid", "sessiondate", "n_done_trials"
)

peh_blobs = (bdata.ParsedEvents * bdata.Sessions & subject_session_key).fetch('peh', as_dict=True)

trial_filter = (trials != 0) & (trials != 1)
pd_blobs, peh_blobs, trials, sess_ids, dates = drop_sessions(
    pd_blobs, peh_blobs, trials, sess_ids, dates, trial_filter
)


pd_dicts = convert_to_dict(pd_blobs)
reformat_pd_dict(pd_dicts)

peh_dicts = convert_to_dict(peh_blobs)

assert len(pd_dicts) == len(peh_dicts) == len(trials) == len(sess_ids) == len(dates) , (
    "number of sessions does not match, assumptions of code below are broken!"
)

peh_dicts, peh_dicts, trials, sess_ids, dates = find_and_fix_len_errors(
    pd_dicts, peh_dicts, trials, sess_ids, dates
)


peh_dicts_for_df = get_peh_vars_for_df(peh_dicts, trials, sess_ids)

df = generate_trials_df(animal_id, pd_dicts, peh_dicts_for_df, trials, sess_ids, dates)



dropping 1 sessions of 93
dropping 0 sessions of 92


In [30]:
df

,trial,animal_id,date,sess_id,hits,violations,temperror,sides,result,sa,...,stage,is_match,sound_pair,early_spoke,t_start,t_end,cpoke_in,cpoke_out,spoke_in,go_time
0,1,R600,2022-07-19,879484,1,0,0,l,1,3.0,...,1,True,"3.0, 3.0",False,90.210404,966.849404,959.323004,959.553604,960.810804,960.423004
1,2,R600,2022-07-19,879484,1,0,0,r,1,12.0,...,1,False,"12.0, 3.0",False,967.863904,1153.647004,1139.392304,1141.228404,1147.608304,1139.892304
2,3,R600,2022-07-19,879484,1,0,0,l,1,3.0,...,1,True,"3.0, 3.0",False,1153.647167,1631.259767,1624.421067,1624.558167,1625.221167,1624.921067
3,4,R600,2022-07-19,879484,1,0,0,l,1,12.0,...,1,True,"12.0, 12.0",False,1631.616704,1775.781804,1768.695904,1768.873004,1769.743204,1769.195904
4,5,R600,2022-07-19,879484,1,0,0,l,1,12.0,...,1,True,"12.0, 12.0",False,1776.272404,2168.973104,2155.757704,2157.015304,2162.934504,2156.257704
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377,378,R600,2022-10-25,893479,<NA>,1,<NA>,l,3,12.0,...,8,True,"12.0, 12.0",True,3838.405298,3859.390598,3853.413498,3853.572898,3854.203498,3854.273498
378,379,R600,2022-10-25,893479,<NA>,1,<NA>,l,3,3.0,...,8,True,"3.0, 3.0",True,3859.390697,3873.848997,3867.669597,3867.810497,3868.456597,3868.529597
379,380,R600,2022-10-25,893479,0,0,0,l,2,3.0,...,8,True,"3.0, 3.0",False,3873.849098,3880.677998,3874.041398,3874.203298,3875.307098,3874.901398
380,381,R600,2022-10-25,893479,0,0,0,l,2,12.0,...,8,True,"12.0, 12.0",False,3880.678099,3887.851299,3881.673599,3881.787499,3882.574699,3882.533599


In [3]:

# #### SINGLE SESSION ####
# session_key = {'sessid': 884257}

# prb = (bdata.Sessions & session_key).fetch(
#     "protocol_data", as_dict=True
# )

# s_id, d, t = (bdata.Sessions & session_key).fetch(
#     "sessid", "sessiondate", "n_done_trials"
# )
# prd = convert_to_dict(prb)
# pd_prepare_dict_for_df(prd)
# session_key = {'sessid': s_id[0]}
# pehb = (bdata.ParsedEvents & session_key).fetch("peh", as_dict=True)
# pehd = bt.transform_blob(pehb[0]['peh'])

